In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
import pandas as pd

from exp.exp_informer import Exp_Informer
import matplotlib.pyplot as plt

from utils.masking import TriangularCausalMask, ProbMask
from models.encoder import Encoder, EncoderLayer, ConvLayer, EncoderStack
from models.decoder import Decoder, DecoderLayer
from models.attn import FullAttention, ProbAttention, AttentionLayer
from models.embed import DataEmbedding, PositionalEmbedding, TokenEmbedding, FixedEmbedding, TemporalEmbedding, TimeFeatureEmbedding

from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

from utils.tools import StandardScaler
from utils.timefeatures import time_features

from data.data_loader import Dataset_ETT_hour, Dataset_ETT_minute, Dataset_Custom, Dataset_Pred
from exp.exp_basic import Exp_Basic
from models.model import Informer, InformerStack

from utils.tools import EarlyStopping, adjust_learning_rate
from utils.metrics import metric

from scipy.special import softmax
import torch
import pandas as pd
import numpy as np
import math

from ray import tune, air
from ray.tune import ResultGrid
from ray.tune.trainable import 

# Additional packages
# Create dataset
df_raw = pd.read_csv('data/processed/SRL/SRL_NEG_00_04.csv')

train =  Dataset_Custom('data/processed/SRL/', freq= 'd', data_path='SRL_NEG_00_04.csv', target='capacity_price')
test =  Dataset_Custom('data/processed/SRL/', freq= 'd', data_path='SRL_NEG_00_04.csv', target='capacity_price',flag='test')
vali =  Dataset_Custom('data/processed/SRL/', freq= 'd', data_path='SRL_NEG_00_04.csv', target='capacity_price', flag='val')

In [2]:
experiment_path = f"ray_tune/tune_SRL_NEG_00_04_24-02-2023_06-56-42/"
print(f"Loading results from {experiment_path}...")

restored_tuner = tune.Tuner.restore(experiment_path)
result_grid = restored_tuner.get_results()

best = result_grid.get_best_result(metric='loss', mode='min')
cpoint = best.checkpoint
a = best.checkpoint.to_dict()

Loading results from ray_tune/tune_SRL_NEG_00_04_24-02-2023_06-56-42/...


In [22]:
np.random.randint(10000)

1069